In [1]:
%pprint #use this to change print sye

Pretty printing has been turned OFF


In [2]:
import random

In [3]:
def return_type_n(file):
    import csv
    with open('reuc_100', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ') #, quotechar='|')
        for i, row in enumerate(reader):
            if i ==0:
                Type=row[0]
            elif i ==1:
                n=int(row[0])
            else:
                break
    return (Type,n)
            

In [4]:
def distmatrix(N):
    import csv
    DIST=[]
    with open('reuc_100', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ') #, quotechar='|')
        for i, row in enumerate(reader):
            if i >= 2+N:
                DIST.append(list(map(float, row)))
        return (DIST)
    return(999999999999999999)
            

In [5]:
def closest(D, n):
    closest = []
    for i in range(n):
        dlist = [[D[i][j] ,j] for j in range(n) if j != i]#generates a list of neighbours sorted with distance from i to j
        dlist.sort()
        closest.append(dlist)
    return closest

In [6]:
def length(tour,D,N):
    z = dist(tour[0][-1], tour[0][0],D)    # edge from last to first city of the tour
    z+=sum(tour[1])      # add length of edge from city i-1 to i
    return z


In [7]:
def is_in(j,LIST):
    for i in LIST:
        if j==i:
            return True
    return False

In [8]:
def return_nearest(i,A,unvisited):
    j=0
    #print(len(A))
    while j < len(A):
        c=A[i][j][1]
        if is_in(c,unvisited):
            return A[i][j]
        else:
            j+=1
    return[999]
    

In [9]:
def create_tour(D,i,N):
    unvisited=list(range(N))
    A=closest(D,N)
    length=[]
    prev=i
    Tour=[i]
    unvisited.remove(i)
    while unvisited!=[]:
        temp=return_nearest(prev,A,unvisited)
        next=temp[1]
        length.append(temp[0])
        Tour.append(next)
        unvisited.remove(next)
        prev=next
    cost=sum(length)
    cost=cost+D[next][i]
    return (Tour,cost)

In [10]:
def create_rand_tour(D,i,N):
    unvisited=list(range(N))
    length=[]
    prev=i
    Tour=[i]
    unvisited.remove(i)
    while unvisited!=[]:
        temp=random.choice(unvisited)
        next=temp
        Tour.append(next)
        unvisited.remove(next)
        prev=next
    rcost=cost(Tour,D)
    return (Tour,rcost)
    

In [11]:
def cost(Tour,D):#use only for GA created Tours with no added distance sum
    rcost=0
    lenght=len(Tour)
    for i in range(lenght-1):
        rcost=rcost+D[Tour[i]][Tour[i+1]]
        #print(rcost)
    rcost=rcost+D[Tour[lenght-1]][Tour[0]]
    return rcost

In [12]:
def sortsol(l):
    l.sort(key = lambda x: x[1])  
    return l

In [13]:
def fitness(sol):
    sum=0
    fitness_sum=0
    fitness=[]
    for i in range(len(sol)):
        fitness_sum=fitness_sum+(1/sol[i][1])**2
    for j in range(len(sol)):
        a=((1/(sol[j][1])**2)/(fitness_sum))
        fitness.append(a)
    return fitness
    

In [14]:
def mk_parent(sol,Fitness,bestsize):
    parents=[]
    #for i in range(bestsize):
        #parents.append(sol[i][0])
    psum=0
    i=0
    for i in range(0, len(sol)): #- bestsize):
        a=random.uniform(0,1)
        psum=0
        i=0
        while True:
            psum+=Fitness[i]
            if psum>=a:
                parents.append(sol[i][0])
                break
            i+=1
    return parents


In [15]:
def make_babies(parent1, parent2):
    child = []
    child_1 = []
    half1_1=[]
    half1_2=[]
    child_2 = []
    half2_1=[]
    half2_2=[]
    
    splicepoint1 = int(random.random() * len(parent1))
    splicepoint2 = int(random.random() * len(parent1))
    
    startsplice = min(splicepoint1, splicepoint2)
    endsplice = max(splicepoint1, splicepoint2)

    for i in range(startsplice, endsplice):
        half1_1.append(parent1[i])
        #print(parent2)
        half2_1.append(parent2[i])
        
    half1_2 = [j for j in parent2 if j not in half1_1]
    half2_2 = [j for j in parent1 if j not in half2_1]

    child_1 = half1_1+half1_2
    child_2 = half2_1+half2_2
    return [child_1,child_2]

In [16]:
def babies(parents, bestsize):
    children = []
    length = len(parents) - bestsize
    parentset = random.sample(parents, len(parents))

    #for i in range(0,bestsize):
        #children.append(parents[i])
    
    for i in range(0, length):
        child = make_babies(parentset[i], parentset[len(parents)-i-1])
        children.append(child[0])
        children.append(child[1])
    return children

In [17]:
def mutate(route,rate):
    for i in range(len(route)):
        #if (random.uniform(0,1) < rate):
        if (random.uniform(0,1) < rate):
            j=int(random.random()*len(route))
            
            cityA=route[i]
            cityB=route[j]
            
            route[i]=cityB
            route[j]=cityA
    return route

In [18]:
def mutate1(route,rate):
    route1=route[:]#mutable list for repeated mutation of same elemet
    for i in range(len(route)):
        #if (random.uniform(0,1) < rate):
        if (random.random() < rate):
            j=int(random.random()*len(route))
            
            cityA=route[i]
            cityB=route[j]
            
            route1[i]=cityB
            route1[j]=cityA
    return route1

In [19]:
def make_sol(tempsol):
    A=[]
    costtemp=0
    for i in range(len(tempsol)):
        costtemp=cost(tempsol[i],D)
        A.append((tempsol[i],costtemp))
    return A

In [20]:
def mutate_children(children, rate):
    mutated=[]
    for i in range(len(children)):
        mutatedchild=mutate(children[i],rate)
        mutated.append(mutatedchild)
    return mutated


In [21]:
def genetic_algorithm(D,N,genrations,rate,bestsize=2):
    import random

    sol=[]
    solution=[]
    bestsize=2
    performance=[]
    for i in range(N):
        T=create_tour(D,i,N)
        sol.append(T)
        sol=sortsol(sol)
    solution=sol[:]
    performance.append((sol[0][1],0))
    
        
    for i in range(genrations):
        selected=solution[0:int(N/2)]
        R=fitness(selected)
        parents=mk_parent(selected,fitness(selected),bestsize)
        children=babies(parents,bestsize)
        solset=mutate_children(children,rate)    
        solution=make_sol(solset)
        for j in range(0,bestsize):
            solution.append(selected[j])
        solution=sortsol(solution)
        #print(solution)
        best=solution[0]
        performance.append((best[1],i))
        i+=1
    return (best,performance)
        
        

In [22]:
def edge_exchange(route,D):
    best = route
    improved = True
    while improved:
        improved = False
        for a in range(1, len(route) - 2):
            for b in range(a + 1, len(route)):
                if b - a == 1: continue  # changes nothing, skip then
                newroute = route[:]
                newroute[a:b] = route[b - 1:a - 1:-1] #swap routes
                if cost(newroute,D) < cost(best,D):
                    best = newroute
                    improved = True
                    route = best
    return best

In [25]:
def genetic_algorithmfinal(D,N,genrations,rate,bestsize=2):
    import random

    sol=[]
    solution=[]
    performance=[]
    for i in range(N):
        B=create_tour(D,i,N)
        #T=edge_exchange(D,B)
        #sol.append((T,cost(T,D)))
        sol.append(B)
    sol=sortsol(sol)
    solution=sol[:]
    performance.append((sol[0][1],0))
    
        
    for i in range(genrations):
        selected=solution[0:25]
        R=fitness(selected)
        parents=mk_parent(selected,fitness(selected),bestsize)
        children=babies(parents,bestsize)
        solset=mutate_children(children,rate)
        solved=[]
        for k in range(len(children)):
            solved.append(edge_exchange(solset[k],D))
        solution=make_sol(solved)
        for j in range(0,bestsize):
            solution.append(selected[j])
        solution=sortsol(solution)
        #print(solution)
        best=solution[0]
        performance.append((best[1],i))
        i+=1
        print((best[1],i),'\n')
    return (best,performance)
        
        

In [26]:
import numpy as np
N=return_type_n(1)[1]
D=distmatrix(N)
result=genetic_algorithmfinal(D,N,1000,0.1)

(1580.2005305080806, 1) 

(1580.2005305080806, 2) 

(1580.2005305080806, 3) 

(1580.2005305080806, 4) 

(1555.72222487422, 5) 

(1555.72222487422, 6) 

(1555.72222487422, 7) 

(1555.72222487422, 8) 

(1555.72222487422, 9) 

(1555.72222487422, 10) 

(1555.72222487422, 11) 

(1555.72222487422, 12) 

(1541.9429735889503, 13) 

(1541.9429735889503, 14) 

(1541.9429735889503, 15) 

(1541.9429735889503, 16) 

(1541.9429735889503, 17) 

(1541.9429735889503, 18) 

(1541.9429735889503, 19) 

(1541.9429735889503, 20) 

(1541.9429735889503, 21) 

(1541.9429735889503, 22) 

(1541.9429735889503, 23) 

(1541.9429735889503, 24) 

(1541.9429735889503, 25) 

(1541.9429735889503, 26) 

(1541.9429735889503, 27) 

(1541.9429735889503, 28) 

(1541.9429735889503, 29) 

(1541.9429735889503, 30) 

(1541.9429735889503, 31) 

(1541.9429735889503, 32) 

(1541.9429735889503, 33) 

(1541.9429735889503, 34) 

(1541.9429735889503, 35) 

(1541.9429735889503, 36) 

(1541.9429735889503, 37) 

(1541.9429735889503, 38) 


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
>>> from math import log
>>> testList2 = [(elem1, elem2) for elem2, elem1 in r[1]]
>>> zip(*testList2)
>>> plt.scatter(*zip(*testList2))
>>> plt.show()

In [ ]:
import numpy as np
N=return_type_n(1)[1]
D=distmatrix(N)
result=genetic_algorithmfinal(D,N,1000,0.0)

In [ ]:
import matplotlib.pyplot as plt
>>> from math import log
>>> testList2 = [(elem1, elem2) for elem2, elem1 in r0[1]]
>>> zip(*testList2)
>>> plt.scatter(*zip(*testList2))
>>> plt.show()

In [ ]:
r[1]

In [ ]:
ra[0]

In [ ]:
ra[9][1]

In [ ]:
ax = plt.gca()


ax.scatter([1, 2, 3], [3, 2, 1], color="b",label='00')

ax.scatter([1.5, 2.5, 3.5], [1, 2, 3], color="r",label='aa')




In [ ]:
edge_exchange(list(range(0,100)),D)

In [ ]:
def makepoints(a):
    import csv
    point=[]
    pointx=[]
    pointy=[]
    pointlabel=[]
    with open('reuc_100', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ') #, quotechar='|')
        for i, row in enumerate(reader):
            if i>1 and i<N+2:
                point.append((float(row[0]),float(row[1])))
                pointx.append(float(row[0]))
                pointy.append(float(row[1]))
                pointlabel.append(str(i))
    return(pointx,pointy,point,pointlabel)

In [ ]:
def make_pairs(tour):
    opa=[]
    for i in range(len(tour[0])-1):
        opa.append((tour[0][i],tour[0][i+1]))
    opa.append((tour[0][i+1],tour[0][0]))
    return opa

In [ ]:
import networkx as nx
import numpy as np


pos = makepoints('a')[2]#coordinates, (x, y) for 10 nodes
connect = make_pairs((r0[0])) #random connections
#print(connect)
#creation of the graph
graph = nx.Graph()
#adding nodes/connections in the graph
for node in range(len(pos)):
    graph.add_node(node)
graph.add_edges_from(connect)

#plot of the nodes using the (x,y) pairs as coordinates
nx.draw(graph, [(x,y) for x,y in pos], node_size=50)

In [ ]:
a=two_opt(D,r0[0][0])

In [ ]:
c=cost(a,D)

In [ ]:
b=(a,c)

In [ ]:
b

In [ ]:
r0[0][1]

In [ ]:
import networkx as nx
import numpy as np


pos = makepoints('a')[2]#coordinates, (x, y) for 10 nodes
connect = make_pairs((b)) #random connections
#print(connect)
#creation of the graph
graph = nx.Graph()
#adding nodes/connections in the graph
for node in range(len(pos)):
    graph.add_node(node)
graph.add_edges_from(connect)

#plot of the nodes using the (x,y) pairs as coordinates
nx.draw(graph, [(x,y) for x,y in pos], node_size=50)

In [ ]:
import networkx as nx
import numpy as np


pos = makepoints('a')[2]#coordinates, (x, y) for 10 nodes
connect = make_pairs((sol[0])) #random connections
#print(connect)
#creation of the graph
graph = nx.Graph()
#adding nodes/connections in the graph
for node in range(len(pos)):
    graph.add_node(node)
graph.add_edges_from(connect)

#plot of the nodes using the (x,y) pairs as coordinates
nx.draw(graph, [(x,y) for x,y in pos], node_size=50)

In [ ]:
A=genetic_algorithmfinal(D,N,1,0.1)

In [ ]:
import matplotlib.pyplot as plt
>>> from math import log
>>> testList2 = [(elem1, elem2) for elem2, elem1 in A[1]]
>>> zip(*testList2)
>>> plt.scatter(*zip(*testList2))
>>> plt.show()

In [ ]:
import networkx as nx
import numpy as np


pos = makepoints('a')[2]#coordinates, (x, y) for 10 nodes
connect = make_pairs(([0, 21, 80, 24, 83, 94, 40, 66, 14, 77, 55, 12, 42, 60, 73, 82, 43, 65, 28, 23, 98, 76, 34, 61, 79, 10, 4, 6, 8, 84, 39, 52, 35, 53, 86, 33, 36, 91, 46, 78, 13, 19, 22, 97, 5, 27, 50, 9, 47, 59, 49, 17, 62, 30, 75, 90, 67, 41, 71, 68, 1, 54, 31, 63, 58, 38, 44, 88, 57, 45, 25, 48, 29, 32, 11, 20, 37, 85, 74, 92, 51, 2, 64, 69, 15, 72, 18, 93, 70, 26, 95, 89, 7, 56, 87, 96, 3, 81, 16, 99],0)) #random connections
#print(connect)
#creation of the graph
graph = nx.Graph()
#adding nodes/connections in the graph
for node in range(len(pos)):
    graph.add_node(node)
graph.add_edges_from(connect)

#plot of the nodes using the (x,y) pairs as coordinates
nx.draw(graph, [(x,y) for x,y in pos], node_size=50)